## Load Master Data Sets and Filter Down to Working Data Sets

The master data sets can be found at https://drive.google.com/drive/u/2/folders/1Ag39uw9kKCKqlEOUkCxcezAutJI6mWyC

This notebook assumes that the datasets it needs are stored locally in the Political_TV_Ad_Archive folder of the
project repository.  The master datasets are not present in the github repository for our project.

In [1]:
# Source datasets are located at https://drive.google.com/drive/u/2/folders/1Ag39uw9kKCKqlEOUkCxcezAutJI6mWyC

import pandas as pd

ads_direct = "../../datasets/Political_TV_Ad_Archive/"

# Read start_time and end_time columns as dates, infer format.
# Drop rows without a location when we read the source advertising dataset.

df = pd.read_csv(ads_direct + "political_ad_entire_dataset.csv").dropna(subset=['location'])

df_sponsors = pd.read_csv(ads_direct + "sponsor_correlation.csv")

In [2]:
# Rename a few columns

df = df.rename({'location': 'region_id',
                'start_time': 'start_utc',
                'end_time': 'end_utc',
               }, 
               axis=1
              )

In [3]:
# We shorten and simplify the region_id values.
# We limit the dataset to rows with the region_ids of interest.

region_ids = ["boston_region",
              "cedar_rapids_region",
              "cleveland_region",
              "denver_region",
              "las_vegas_region",
              "milwaukee_region",
              "new_york_city_region",
              "philadelphia_region",
              "phoenix_region",
              "raleigh_region",
              "san_francisco_region",
              "tampa_region",
              "washington_dc_region",
              ]

df['region_id'] = df['region_id'].str.replace(
    "^Ceder Rapids", "Cedar_Rapids", regex=True)
df['region_id'] = df['region_id'].str.replace(
    "^Las Vegas", "Las_Vegas", regex=True)
df['region_id'] = df['region_id'].str.replace(
    "^New York City", "New_York_City", regex=True)
df['region_id'] = df['region_id'].str.replace(
    "^San Francisco", "San_Francisco", regex=True)
df['region_id'] = df['region_id'].str.replace(
    "^Washington, DC", "Washington_DC", regex=True)
df['region_id'] = df['region_id'].str.replace(
    "[,/ -].*", "_region", regex=True)
df['region_id'] = df['region_id'].str.lower()

df = df[df.region_id.isin(region_ids)]

In [4]:
# We are only looking at the presidential race.
df = df[df.race == "PRES"]

# We are only considering ads that mention Trump or Clinton.
df = df[df.candidates.str.contains("Trump|Clinton", case=False, regex=True)]

# Eliminate that are redundant or otherwise not relevant to our project.
df = df.drop(columns=["id",
                      "wp_identifier",
                      "race",
                      "cycle",
                      "type",
                      "date_created"])

In [5]:
df.dtypes

network          object
region_id        object
program          object
program_type     object
start_utc        object
end_utc          object
archive_id       object
embed_url        object
sponsors         object
sponsor_types    object
subjects         object
candidates       object
message          object
dtype: object

In [6]:
df['start_utc'] = pd.to_datetime(df['start_utc'])
df['end_utc'] = pd.to_datetime(df['end_utc'])

In [7]:
df.dtypes

network                       object
region_id                     object
program                       object
program_type                  object
start_utc        datetime64[ns, UTC]
end_utc          datetime64[ns, UTC]
archive_id                    object
embed_url                     object
sponsors                      object
sponsor_types                 object
subjects                      object
candidates                    object
message                       object
dtype: object

In [8]:
df.shape

(116616, 13)

In [9]:
df_sponsors.columns

Index(['sponsors', 'beneficiary', 'ads_count', 'reference', 'note'], dtype='object')

In [10]:
# The df_sponsor_correlation dataframe identifies the assumed beneficiary for
# each ad that a given sponsor sponsors.  The values are Trump, Clinton, Other
# and Split.  We use that information to identify a beneficiary for each ad and
# also to remove ads where we have no basis for identifying which candidate
# is the beneficiary.  In other words, we will drop ads where the beneficiary
# is not Trump, Clinton or Split.

df_sponsors.beneficiary.value_counts()

Clinton    19
Trump      17
Other       4
Split       1
Name: beneficiary, dtype: int64

In [11]:
# Remove sponsors where the beneficiary is not Trump, CLinton or Split.
# These are the sponsors identified with "Other" as the beneficiary.

df_sponsors = df_sponsors[df_sponsors.beneficiary != "Other"]

In [12]:
df_sponsors.beneficiary.value_counts()

Clinton    19
Trump      17
Split       1
Name: beneficiary, dtype: int64

In [13]:
# An inner merge on the sponsors column will allow us to add a beneficiary for
# each ad where we believe we can identify an intended beneficiary and also,
# as a side effect, will remove ads where we are unable to associate a
# beneficiary

df = pd.merge(df, df_sponsors[['sponsors', 'beneficiary']],
              on='sponsors',
              how='inner')

In [14]:
df.beneficiary.value_counts()

Clinton    70796
Trump      36021
Split        511
Name: beneficiary, dtype: int64

In [15]:
# Internet Archive is the only sponsor where one group of ads benefits Clinton
# and the other Trump.

# id PolAd_DonaldTrump_z40lb benefits Clinton

df.loc[df.archive_id == "PolAd_DonaldTrump_z40lb", "beneficiary"] = "Clinton"

# id PolAd_DonaldTrump_HillaryClinton_xz04u benefits Trump

df.loc[df.archive_id == "PolAd_DonaldTrump_HillaryClinton_xz04u",
       "beneficiary"] = "Trump"

df.beneficiary.value_counts()

Clinton    71119
Trump      36209
Name: beneficiary, dtype: int64

In [16]:
df.region_id.value_counts()

philadelphia_region     13710
las_vegas_region        13550
tampa_region            12982
cleveland_region        12149
san_francisco_region    11285
cedar_rapids_region     10331
boston_region           10276
raleigh_region           7310
denver_region            5581
milwaukee_region         4098
phoenix_region           3242
washington_dc_region     1819
new_york_city_region      995
Name: region_id, dtype: int64

In [17]:
# Cluster regions into time zones.

eastern_tz_regions = ["boston_region",
                      "cleveland_region",
                      "new_york_city_region",
                      "philadelphia_region",
                      "raleigh_region",
                      "tampa_region",
                      "washington_dc_region",
                      ]

central_tz_regions = ["cedar_rapids_region",
                      "milwaukee_region",
                      ]

mountain_tz_regions = ["denver_region",
                       "phoenix_region"
                       ]

pacific_tz_regions = ["las_vegas_region",
                      "san_francisco_region"
                      ]

# Create time zone boolean arrays.

e = df['region_id'].isin(eastern_tz_regions)
c = df['region_id'].isin(central_tz_regions)
m = df['region_id'].isin(mountain_tz_regions)
p = df['region_id'].isin(pacific_tz_regions)

In [19]:
# UTC start_time and end_time values to time zone aware objects.
# Then strip the time zone information from the resulting objects
# using apply and tz.localize(None).  The normal Pandas methods
# do not work when the column has datetime objects from more than one
# time zone.

df.loc[e, 'start_time'] = df.loc[e, 'start_utc'].dt.tz_convert('US/Eastern')
df.loc[c, 'start_time'] = df.loc[c, 'start_utc'].dt.tz_convert('US/Central')
df.loc[m, 'start_time'] = df.loc[m, 'start_utc'].dt.tz_convert('US/Mountain')
df.loc[p, 'start_time'] = df.loc[p, 'start_utc'].dt.tz_convert('US/Pacific')

df.loc[e, 'end_time'] = df.loc[e, 'end_utc'].dt.tz_convert('US/Eastern')
df.loc[c, 'end_time'] = df.loc[c, 'end_utc'].dt.tz_convert('US/Central')
df.loc[m, 'end_time'] = df.loc[m, 'end_utc'].dt.tz_convert('US/Mountain')
df.loc[p, 'end_time'] = df.loc[p, 'end_utc'].dt.tz_convert('US/Pacific')

df['start_time'] = df.apply(lambda row: row['start_time'].tz_localize(None), axis=1)
df['end_time'] = df.apply(lambda row: row['end_time'].tz_localize(None), axis=1)

In [20]:
print(df.loc[e, ['start_time', 'start_utc', 'region_id']].head())
print(df.loc[c, ['start_time', 'start_utc', 'region_id']].head())
print(df.loc[m, ['start_time', 'start_utc', 'region_id']].head())
print(df.loc[p, ['start_time', 'start_utc', 'region_id']].head())
print(df.loc[e, ['end_time', 'end_utc', 'region_id']].head())
print(df.loc[c, ['end_time', 'end_utc', 'region_id']].head())
print(df.loc[m, ['end_time', 'end_utc', 'region_id']].head())
print(df.loc[p, ['end_time', 'end_utc', 'region_id']].head())

           start_time                 start_utc       region_id
1 2016-09-06 17:58:25 2016-09-06 21:58:25+00:00  raleigh_region
3 2016-08-30 13:59:20 2016-08-30 17:59:20+00:00    tampa_region
5 2016-08-30 15:55:14 2016-08-30 19:55:14+00:00    tampa_region
6 2016-08-30 09:22:30 2016-08-30 13:22:30+00:00    tampa_region
7 2016-09-06 20:45:00 2016-09-07 00:45:00+00:00  raleigh_region
            start_time                 start_utc            region_id
26 2016-09-08 09:39:00 2016-09-08 14:39:00+00:00  cedar_rapids_region
30 2016-09-05 12:32:00 2016-09-05 17:32:00+00:00  cedar_rapids_region
36 2016-09-05 23:50:59 2016-09-06 04:50:59+00:00  cedar_rapids_region
38 2016-09-06 17:46:32 2016-09-06 22:46:32+00:00  cedar_rapids_region
39 2016-09-07 15:33:59 2016-09-07 20:33:59+00:00  cedar_rapids_region
              start_time                 start_utc       region_id
1911 2016-09-08 09:04:30 2016-09-08 15:04:30+00:00  phoenix_region
1919 2016-09-08 08:04:29 2016-09-08 14:04:29+00:00   denver_re

In [21]:
df.dtypes

network                       object
region_id                     object
program                       object
program_type                  object
start_utc        datetime64[ns, UTC]
end_utc          datetime64[ns, UTC]
archive_id                    object
embed_url                     object
sponsors                      object
sponsor_types                 object
subjects                      object
candidates                    object
message                       object
beneficiary                   object
start_time            datetime64[ns]
end_time              datetime64[ns]
dtype: object

In [24]:
print(f"Earliest: {df.start_time.min()}")
print(f"Latest: {df.start_time.max()}")

Earliest: 2015-10-05 06:10:49
Latest: 2016-11-29 12:44:37


In [26]:
# Filter the dataframe to ads that ran on or after June 1, 2016 but before
# Novmber 9, 2016.  Election day was November 8, 2016.

df = df[(df.start_time >= "2016-06-01") & (df.start_time <= "2016-11-08")]
print(f"Earliest: {df.start_time.min()}")
print(f"Latest: {df.start_time.max()}")

Earliest: 2016-06-01 08:30:06
Latest: 2016-11-07 23:59:56


In [27]:
# Create date, day_of_week, air_time and duration fields.

df["date"] = df["start_time"].dt.date
df["day_of_week"] = df["start_time"].dt.day_name()
df["air_time"] = df["start_time"].dt.time
df["duration"] = df["end_time"] - df["start_time"]

In [28]:
df.columns

Index(['network', 'region_id', 'program', 'program_type', 'start_utc',
       'end_utc', 'archive_id', 'embed_url', 'sponsors', 'sponsor_types',
       'subjects', 'candidates', 'message', 'beneficiary', 'start_time',
       'end_time', 'date', 'day_of_week', 'air_time', 'duration'],
      dtype='object')

In [29]:
df.dtypes

network                       object
region_id                     object
program                       object
program_type                  object
start_utc        datetime64[ns, UTC]
end_utc          datetime64[ns, UTC]
archive_id                    object
embed_url                     object
sponsors                      object
sponsor_types                 object
subjects                      object
candidates                    object
message                       object
beneficiary                   object
start_time            datetime64[ns]
end_time              datetime64[ns]
date                          object
day_of_week                   object
air_time                      object
duration             timedelta64[ns]
dtype: object

In [30]:
df.shape

(85133, 20)

In [31]:
df.to_csv(
    ads_direct + "political_ad_pres_airing_from_june.gz",
    compression="gzip",
    index=False,
)